In [1]:
#Imports
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#Network parameters
batch_size = 64
img_height = 224
img_width = 224

In [4]:
#Process data
test_folder = '../images/uk_not_uk/test'
preprocessor = keras.applications.resnet50.preprocess_input
test_data_generator = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocessor)
test_set = test_data_generator.flow_from_directory(test_folder, class_mode='binary',shuffle = True,target_size=(img_height,img_width))

Found 374 images belonging to 2 classes.


In [6]:
#Get pretrained model
resnet_model = tf.keras.applications.ResNet50(input_shape=(img_width, img_height, 3), include_top=False, pooling='avg', weights='imagenet')
resnet_model.trainable = False
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [7]:
#compile and set params
#model = keras.models.Sequential([resnet_model, keras.layers.Dense(1, name='logits')])
model = keras.models.Sequential([resnet_model, keras.layers.Dense(1, name='logits'), keras.layers.Activation('sigmoid',name='sigmoid_out')])
opt = keras.optimizers.Adam(learning_rate=0.005)
loss = keras.losses.BinaryCrossentropy()
#checkpoint = ModelCheckpoint('./model_checkpoints/uk_not_uk/more_epochs_location_classifier', save_weights_only=True, monitor="val_loss", save_best_only=True)
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='binary_accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
    keras.metrics.Accuracy(name='accuracy'),
    keras.metrics.MeanAbsoluteError(name='mae')
]

model.compile(loss=loss, optimizer=opt, metrics=METRICS)

In [8]:
model.load_weights('./model_checkpoints/uk_not_uk/location_classifier')

In [9]:
res = model.evaluate(test_set)

12/12 [==============================] - 42s 3s/step - loss: 0.7615 - tp: 30.6154 - fp: 26.4615 - tn: 89.9231 - fn: 73.0000 - binary_accuracy: 0.5587 - precision: 0.5207 - recall: 0.2970 - auc: 0.5725 - accuracy: 0.0000e+00 - mae: 0.4619


In [17]:
import pandas as pd
hist = pd.read_csv('./model_checkpoints/uk_not_uk/history.csv')
hist.head(10)

,Unnamed: 0,loss,tp,fp,tn,fn,binary_accuracy,precision,recall,auc,...,val_tp,val_fp,val_tn,val_fn,val_binary_accuracy,val_precision,val_recall,val_auc,val_accuracy,val_mae
0,0,0.607360,14875.0,6966.0,17034.0,9125.0,0.664771,0.681059,0.619792,0.724288,...,2781.0,415.0,5585.0,3219.0,0.697167,0.870150,0.463500,0.821266,0.0,0.383525
1,1,0.561719,16014.0,5983.0,18017.0,7986.0,0.708979,0.728008,0.667250,0.780652,...,4184.0,1068.0,4932.0,1816.0,0.759667,0.796649,0.697333,0.837554,0.0,0.360385
2,2,0.540536,16487.0,5631.0,18369.0,7513.0,0.726167,0.745411,0.686958,0.802186,...,4000.0,872.0,5128.0,2000.0,0.760667,0.821018,0.666667,0.847191,0.0,0.351955
3,3,0.524037,16791.0,5403.0,18597.0,7209.0,0.737250,0.756556,0.699625,0.817910,...,4227.0,978.0,5022.0,1773.0,0.770750,0.812104,0.704500,0.853143,0.0,0.346838
4,4,0.516342,16986.0,5305.0,18695.0,7014.0,0.743354,0.762012,0.707750,0.824245,...,4515.0,1196.0,4804.0,1485.0,0.776583,0.790580,0.752500,0.856080,0.0,0.339253
5,5,0.507667,17156.0,5266.0,18734.0,6844.0,0.747708,0.765141,0.714833,0.831122,...,3938.0,724.0,5276.0,2062.0,0.767833,0.844702,0.656333,0.858986,0.0,0.331846
6,6,0.501798,17370.0,5196.0,18804.0,6630.0,0.753625,0.769742,0.723750,0.835620,...,3384.0,400.0,5600.0,2616.0,0.748667,0.894292,0.564000,0.861576,0.0,0.329679
7,7,0.494154,17462.0,5058.0,18942.0,6538.0,0.758417,0.775400,0.727583,0.841921,...,4125.0,813.0,5187.0,1875.0,0.776000,0.835358,0.687500,0.863357,0.0,0.321416
8,8,0.490261,17549.0,5041.0,18959.0,6451.0,0.760583,0.776848,0.731208,0.845039,...,4595.0,1149.0,4851.0,1405.0,0.787167,0.799965,0.765833,0.864737,0.0,0.320510
9,9,0.485045,17698.0,4928.0,19072.0,6302.0,0.766042,0.782197,0.737417,0.848992,...,4694.0,1248.0,4752.0,1306.0,0.787167,0.789970,0.782333,0.865733,0.0,0.320877


In [ ]:
hist